# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [229]:
import pandas as pd
import numpy as np


---
## 2. Load data

In [353]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [308]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0,...,0,0,0,2,100,44,2,100,44,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,190,...,0,0,0,0,0,0,0,0,0,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,48,...,0,0,0,0,0,0,0,0,0,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [356]:
price_df = pd.read_csv('price_data (1).csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0,0,0,44,0,0
1,038af19179925da21a25619c5a24b745,2015-02-01,0,0,0,44,0,0
2,038af19179925da21a25619c5a24b745,2015-03-01,0,0,0,44,0,0
3,038af19179925da21a25619c5a24b745,2015-04-01,0,0,0,44,0,0
4,038af19179925da21a25619c5a24b745,2015-05-01,0,0,0,44,0,0


In [310]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0,0
1,0004351ebdd665e6ee664792efc4fd13,-0,0
2,0010bcc39e42b3c2131ed2ce55246e3c,0,2
3,0010ee3855fdea87602a5b7aba8e42de,-0,0
4,00114d74e963e47177db89bc70108537,-0,-0


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

### Removing Columns
We can see that `num_years_antig` is not very useful since we already have start and end date of the customer. Also `margin_gross_pow_ele` is an column with the same values of the column `margin_net_pow_ele`.

In [311]:
columns_to_remove = ['num_years_antig', 'margin_gross_pow_ele']
df = df.drop(columns=columns_to_remove)

In [312]:
print(df.columns)

Index(['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_net_pow_ele', 'nb_prod_act', 'net_margin',
       'origin_up', 'pow_max', 'var_year_price_off_peak_var',
       'var_year_price_peak_var', 'var_year_price_mid_peak_var',
       'var_year_price_off_peak_fix', 'var_year_price_peak_fix',
       'var_year_price_mid_peak_fix', 'var_year_price_off_peak',
       'var_year_price_peak', 'var_year_price_mid_peak',
       'var_6m_price_off_peak_var', 'var_6m_price_peak_var',
       'var_6m_price_mid_peak_var', 'var_6m_price_off_peak_fix',
       'var_6m_price_peak_fix', 'var_6m_price_mid_peak_fix',
       'var_6m_price_off_peak', 'var_6m_price_peak', 'var_6

### Date Feature


Changing the date format can be more useful and easy to analyze

In [ ]:
df['date_activ'] = df['date_activ'].dt.strftime('%B %d %Y')
df['date_end'] = df['date_end'].dt.strftime('%B %d %Y')
df['date_modif_prod'] = df['date_modif_prod'].dt.strftime('%B %d %Y')
df['date_renewal'] = df['date_renewal'].dt.strftime('%B %d %Y')

price_df["price_date"] = price_df['price_date'].dt.strftime('%B %d %Y')

In [359]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn,customer_duration
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,June 15 2013,June 15 2016,November 01 2015,June 23 2015,0,...,0,0,2,100,44,2,100,44,1,1096 days
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,August 21 2009,August 30 2016,August 21 2009,August 31 2015,190,...,0,0,0,0,0,0,0,0,0,2566 days
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,April 16 2010,April 16 2016,April 16 2010,April 17 2015,48,...,0,0,0,0,0,0,0,0,0,2192 days


In [315]:
price_df.head(3)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,January 01 2015,0,0,0,44,0,0
1,038af19179925da21a25619c5a24b745,February 01 2015,0,0,0,44,0,0
2,038af19179925da21a25619c5a24b745,March 01 2015,0,0,0,44,0,0


### Calculate churn rate
Analyzing the churn rate for different electricity campaign codes that the customer first subscribed to.

In [316]:
# Group by 'origin_up' and calculate churn rate
origin_up_churn = df.groupby(['origin_up'])['churn'].mean().reset_index()
# Calculate churn percentage
origin_up_churn['churn_percentage'] = origin_up_churn['churn'] * 100

origin_up_churn = origin_up_churn.sort_values(by='churn_percentage', ascending=False)

origin_up_churn.head()

,origin_up,churn,churn_percentage
4,lxidpiddsbxsbosboudacockeimpuepw,0,13
3,ldkssxwpmemidmecebumciepifcamkci,0,8
0,MISSING,0,6
2,kamkkxfxxuwbdslkwifmmcsiusiuosws,0,6
1,ewxeelcelemmiwuafmddpobolfuxioce,0,0


We can see that orgin_up `lxidpiddsbxsbosboudacockeimpuepw` has the most churn rate.

### Combining consumptions
We can get the gas consumption of the last month and get the average consumption per channel of sales.

In [317]:
df['cons_gas_last_month'] = df['cons_gas_12m'] / 12
df['total_cons_last_month'] = df['cons_gas_last_month'] + df['cons_last_month']
df['total_cons_12m'] = df['cons_gas_12m'] + df['cons_12m']

grouped_data = df.groupby(['channel_sales']).agg({
    'total_cons_last_month': 'mean',
    'total_cons_12m': 'mean',
}).reset_index()


In [318]:
grouped_data.head()

,channel_sales,total_cons_last_month,total_cons_12m
0,MISSING,14350,140920
1,epumfxlbckeskwekxbiuasklxalciiuu,1626,23079
2,ewpakwlliwisiwduibdlfmalxowmwpci,4316,49493
3,fixdbufsefwooaasfcxdxadsiekoceaa,176318,1492222
4,foosdfpfkusacimwkcsosbicdxkicaua,11668,122059


We see above that the channel `epumfxlbckeskwekxbiuasklxalciiuu` has the least consumption which could lead us predicting that customers are not actively using this service and more likely to churn. 

### Ratio of `cons_last_month` to `cons_12m`

Using ratios helps me examine how much customer's electricity use changes over time. This helps us see patterns that could be important to predicting if the customer will churn and how will the customer act over time.

In [319]:
monthly_cons_by_id = df.groupby(['id', 'channel_sales']).agg({'cons_last_month': 'mean', 'cons_12m': 'mean'}).reset_index()
monthly_cons_by_id['ratio_consumption'] = monthly_cons_by_id['cons_last_month'].divide(monthly_cons_by_id['cons_12m'])
ratio_res = monthly_cons_by_id[['id', 'channel_sales', 'cons_last_month', 'cons_12m','ratio_consumption']]

In [320]:
ratio_res.head()

,id,channel_sales,cons_last_month,cons_12m,ratio_consumption
0,0002203ffbb812588b632b9e628cc38d,foosdfpfkusacimwkcsosbicdxkicaua,3084,22034,0
1,0004351ebdd665e6ee664792efc4fd13,MISSING,0,4060,0
2,0010bcc39e42b3c2131ed2ce55246e3c,usilxuppasemubllopkaafesmlibmsdf,1062,7440,0
3,00114d74e963e47177db89bc70108537,ewpakwlliwisiwduibdlfmalxowmwpci,0,11272,0
4,0013f326a839a2f6ad87a1859952d227,foosdfpfkusacimwkcsosbicdxkicaua,19394,267414,0


### Contract Duration
This feature represents the duration of the customer contract. It provides insights of the customer loyalty, and of how much a customer is more committed.

In [354]:
df['customer_duration'] = (df['date_end'] - df['date_activ'])

customer_duration_by_id = df.groupby(['id']).agg({'customer_duration': 'mean'}).reset_index()

In [322]:
customer_duration_by_id.head()

,id,customer_duration
0,0002203ffbb812588b632b9e628cc38d,2224 days
1,0004351ebdd665e6ee664792efc4fd13,2511 days
2,0010bcc39e42b3c2131ed2ce55246e3c,1165 days
3,00114d74e963e47177db89bc70108537,2192 days
4,0013f326a839a2f6ad87a1859952d227,1095 days


### Combining datasets

In [369]:
df = df.groupby(['channel_sales', 'id']).agg({
    'total_cons_last_month': 'mean',
    'total_cons_12m': 'mean',
}).reset_index()
merged_df = pd.merge(df, price_df, on='id', how='left')

merged_df.head()

KeyError: "Column(s) ['total_cons_12m', 'total_cons_last_month'] do not exist"